# Basic indexing and searching with RAGatouille

In this quick example, we'll use the `RAGPretrainedModel` magic class to demonstrate how to:

- **Build an index from raw documents**
- **Search an index for relevant documents**
- **Load an index and the associated pretrained model to update or query it.**

Please note: Indexing is currently not supported on Google Colab and Windows 10.

First, let's load up a pre-trained ColBERT model:

In [1]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

[Apr 01, 08:34:33] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


And that's all you need to do to load the model! All the config is now stored, and ready to be used for indexing.

## Creating an index

Let's index some documents now. We'll use data from Wikipedia, to build our Miyazaki-Index, which will store all you could ever know about Hayao Miyazaki('s wikipedia page).

First, let's write a function to fetch the data from the Wikipedia with a clear user-agent, to be a good netizen:

In [4]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.
    
    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {
        "User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"
    }

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data['query']['pages'].values()))
    return page['extract'] if 'extract' in page else None

And now, let's use it to fetch the page's content and check how long it is:

In [5]:
full_document = get_wikipedia_page("Hayao_Miyazaki")
len(full_document)

45680

In [8]:
print(full_document[:500])

Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.
Born in Tokyo City in the Empire of Japan, Miyazaki expressed interest in manga and animation from an early age


That's a lot of characters! Thankfully, `RAGPretrainedColBERT.index()` also relies on a `CorpusProcessor`! It takes in various pre-processing functions and applies them to your documents before embedding and indexing them.

By default, `CorpusProcessor` uses LlamaIndex's `SentenceSplitter`, with a chunk-size defined by your index's max document length. By default, `max_document_length` is 256 tokens, but you can set it to whatever you like.

Let's keep our information units small and go for 180 when creating our index. We'll also add an optional document ID and an optional metadata entry for our index:

In [9]:
RAG.index(
    collection=[full_document], 
    document_ids=['miyazaki'],
    document_metadatas=[{"entity": "person", "source": "wikipedia"}],
    index_name="Miyazaki", 
    max_document_length=180, 
    split_documents=True
    )

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 01, 08:39:12] #> Creating directory .ragatouille/colbert/indexes/Miyazaki 


[Apr 01, 08:39:15] [0] 		 #> Encoding 81 passages..


/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
 33%|███▎      | 1/3 [00:10<00:20, 10.42s/it]/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 3/3 [00:20<00:00,  6.69s/it]

[Apr 01, 08:39:35] [0] 		 avg_doclen_est = 129.88888549804688 	 len(local_sample) = 81
[Apr 01, 08:39:35] [0] 		 Creating 1,024 partitions.
[Apr 01, 08:39:35] [0] 		 *Estimated* 10,520 embeddings.
[Apr 01, 08:39:35] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki/plan.json ..


used 15 iterations (0.7459s) to cluster 9995 items into 1024 clusters
[0.037, 0.041, 0.038, 0.034, 0.031, 0.037, 0.034, 0.037, 0.033, 0.034, 0.037, 0.038, 0.035, 0.037, 0.034, 0.04, 0.032, 0.032, 0.034, 0.035, 0.038, 0.039, 0.037, 0.035, 0.039, 0.032, 0.038, 0.032, 0.038, 0.035, 0.036, 0.037, 0.039, 0.033, 0.033, 0.034, 0.035, 0.032, 0.034, 0.04, 0.034, 0.038, 0.037, 0.032, 0.04, 0.035, 0.039, 0.035, 0.036, 0.037, 0.036, 0.036, 0.034, 0.039, 0.037, 0.037, 0.04, 0.038, 0.041, 0.031, 0.035, 0.034, 0.034, 0.033, 0.038, 0.038, 0.036, 0.04, 0.032, 0.032, 0.035, 0.036, 0.034, 0.035, 0.035, 0.032, 0.035, 0.038, 0.038, 0.036, 0.037, 0.039, 0.036, 0.04, 0.034, 0.037, 0.038, 0.037, 0.034, 0.04, 0.035, 0.037, 0.034, 0.035, 0.035, 0.037, 0.038, 0.037, 0.035, 0.036, 0.04, 0.039, 0.033, 0.035, 0.037, 0.035, 0.036, 0.034, 0.037, 0.036, 0.034, 0.037, 0.038, 0.032, 0.037, 0.037, 0.035, 0.036, 0.036, 0.039, 0.036, 0.035, 0.034, 0.033, 0.032, 0.037, 0.038, 0.032]


0it [00:00, ?it/s]

...

100%|██████████| 3/3 [00:16<00:00,  5.59s/it]
1it [00:16, 16.87s/it]
100%|██████████| 1/1 [00:00<00:00, 704.21it/s]

[Apr 01, 08:39:53] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 01, 08:39:53] #> Building the emb2pid mapping..
[Apr 01, 08:39:53] len(emb2pid) = 10521



100%|██████████| 1024/1024 [00:00<00:00, 50675.69it/s]

[Apr 01, 08:39:53] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/Miyazaki'

And that's our index created! It's already compressed and save to disk, so you're ready to use it anywhere you want. By the way, the default behaviour of `index()` is to split documents, but if for any reason you'd like them to remain intact (if you've already preprocessed them, for example), you can set it to false to bypass it!

Let's move on to querying our index now...

## Retrieving Documents

`RAGPretrainedModel` has just indexed our document, so the index is already loaded into it and ready to use! 

Searching is very simple and straightforward, let's say I have a single query:

In [10]:
k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
results = RAG.search(query="What animation studio did Miyazaki found?", k=k)
results

Loading searcher for index Miyazaki for the first time... This may take a few seconds
[Apr 01, 08:40:18] #> Loading codec...
[Apr 01, 08:40:18] #> Loading IVF...
[Apr 01, 08:40:18] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[Apr 01, 08:40:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 1776.49it/s]

[Apr 01, 08:40:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 240.10it/s]

[Apr 01, 08:40:32] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Apr 01, 08:40:45] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What animation studio did Miyazaki found?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  7284,  2996,  2106,  2771,  3148, 18637,  2179,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': 'In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".',
  'score': 25.903501510620117,
  'rank': 1,
  'document_id': 'miyazaki',
  'passage_id': 28,
  'document_metadata': {'entity': 'person', 'source': 'wikipedia'}},
 {'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature fi

But is it efficient? Let's check how long it takes ColBERT to embed our query and retrieve documents. Because ColBERT's main retrieval approach relies on `maxsim`, a very efficient operation, searching through orders of magnitudes more documents shouldn't take much longer:

In [6]:
%%timeit
RAG.search(query="What animation studio did Miyazaki found?")

52.8 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


You can also batch queries, which will run faster if you've got many different queries to run at once. The output format is the same as for a single query, except it's a list of lists, where item at index `i` will correspond to the query at index `i`:

In [7]:
all_results = RAG.search(query=["What animation studio did Miyazaki found?", "Miyazaki son name"], k=k)
all_results

2it [00:00, 105.94it/s]


[[{'content': 'In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".',
   'score': 25.90448570251465,
   'rank': 1,
   'document_id': 'miyazaki',
   'document_metadata': {'entity': 'person', 'source': 'wikipedia'}},
  {'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is wid

And that's it for the basics of querying an index! You're now ready to index and retrieve documents with RAGatouille!

## Using an already-created index

In the examples above, we embedded documents into an index and queried it during the same session. But a key feature is **persistence**: indexing is the slowest part, we don't want to have to do this every-time!

Loading an already-created Index is just as straightforward as creating one from scratch. First, we'll load up an instance of RAGPretrainedModel from the index, where the full configuration of the embedder is stored:

In [8]:
# This is the path to index. We recommend keeping this path format when using RAGatouille somewhere else.
path_to_index = ".ragatouille/colbert/indexes/Miyazaki/"
RAG = RAGPretrainedModel.from_index(path_to_index)

/Users/bclavie/miniforge3/envs/ragatouille/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


And that's it! The index is now fully ready to be queried using `search()` as above.

### Updating an index

Once you've loaded an existing index, you might want to add new documents to it. RAGatouille supports this via the `RAGPretrainedModel.add_to_index()` function. Due to the way ColBERT stores documents as bags-of-embeddings, there are cases where recreating the index is more efficient than updating it -- you don't need to worry about it, the most efficient method is automatically used when you call `add_to_index()`.

You want to expand, and cover more of Studio Ghibli, so let's get the Studio's page into our index too!

In [11]:
new_documents = get_wikipedia_page("Studio_Ghibli")

RAG.add_to_index([new_documents])

[Apr 01, 08:45:34] #> Loading codec...
[Apr 01, 08:45:34] #> Loading IVF...
[Apr 01, 08:45:34] #> Loading doclens...


/Users/abhi/miniconda3/envs/RAG/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 1003.18it/s]

[Apr 01, 08:45:34] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 196.97it/s]

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 01, 08:45:34] #> Note: Output directory .ragatouille/colbert/indexes/Miyazaki already exists




...

[Apr 01, 08:46:09] [0] 		 #> Encoding 136 passages..


100%|██████████| 5/5 [00:33<00:00,  6.67s/it]
1it [00:33, 33.60s/it]
100%|██████████| 1/1 [00:00<00:00, 780.63it/s]

[Apr 01, 08:46:42] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 01, 08:46:42] #> Building the emb2pid mapping..
[Apr 01, 08:46:42] len(emb2pid) = 17408



100%|██████████| 2048/2048 [00:00<00:00, 43656.25it/s]

[Apr 01, 08:46:42] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki/ivf.pid.pt
Successfully updated index with 55 new documents!
 New index size: 136


And again, that's it! The index has been updated with your new document set, and the updates are already persisted to disk. You're now ready to query it with `search()`!